In [91]:
import torch
from torch import nn
from torch.autograd import Variable
import pandas as pd
import dateutil
import datetime
from sklearn.model_selection import train_test_split
import numpy as np
import torch.nn.functional as funcs

In [92]:
# 68,335,729 data points in total
import random
p = 0.0001  # 0.01% of the lines -> 6800 data points
# keep the header, then take only 1% of lines
# if random from [0,1] interval is greater than 0.01 the row will be skipped
"""
df = pd.read_csv(
         'data/all_events.csv',
         header=0, 
         skiprows=lambda i: i>0 and random.random() > p
)
"""
# df = pd.read_csv('../data/Kiasma_sorted.csv', nrows=500000, low_memory=False)
# df = pd.read_csv('../data/Rautatientoriitä_sorted.csv', low_memory=False)
# df = pd.read_csv('../data/AaltoyliopistoMKorkeakouluaukio_sorted.csv', low_memory=False)
df = pd.read_csv('../data/Arabiankauppakeskus_sorted.csv', low_memory=False)

## Drop duplicated based on timestamp

In [93]:
df = df.drop_duplicates('ts').reset_index()

In [94]:
df.shape

(320175, 9)

## Add some useful columns

In [95]:
# sort based on timestamp
df = df.sort_values('ts')
# the hour and minutes of the timestamp
df['hour_and_minutes'] = df['ts'].apply(lambda x: x[11:16])
# month of timestamp
df['month'] = df['ts'].apply(lambda x: x[5:7])
# the percentage of bikes
df['bikes_percent'] = df['bikes'] / df['total_slots']
df['minutes_from_midnight'] = df['ts'].apply(lambda x: int(x[11:13]) * 60 + int(x[14:16]))
df['hour'] = df['hour_and_minutes'].apply(lambda x: x[:2])
df['date'] = df['ts'].apply(pd.to_datetime)
df['weekday'] = df['date'].apply(lambda x: x.weekday())
df['weekend'] = df['weekday'].apply(lambda x: x > 4)
df.head()

,index,ts,lat,lon,operative,sid,name,bikes,total_slots,hour_and_minutes,month,bikes_percent,minutes_from_midnight,hour,date,weekday,weekend
0,0,2017-03-29T08:26:01,60.202791,24.966613,1,D21,Arabian kauppakeskus,0,1,08:26,03,0.0,506,08,2017-03-29 08:26:01,2,False
1,1,2017-03-29T08:27:01,60.202791,24.966613,1,D21,Arabian kauppakeskus,0,1,08:27,03,0.0,507,08,2017-03-29 08:27:01,2,False
2,2,2017-03-29T08:28:01,60.202791,24.966613,1,D21,Arabian kauppakeskus,0,1,08:28,03,0.0,508,08,2017-03-29 08:28:01,2,False
3,3,2017-03-29T08:29:01,60.202791,24.966613,1,D21,Arabian kauppakeskus,0,1,08:29,03,0.0,509,08,2017-03-29 08:29:01,2,False
4,4,2017-03-29T08:30:01,60.202791,24.966613,1,D21,Arabian kauppakeskus,0,1,08:30,03,0.0,510,08,2017-03-29 08:30:01,2,False


## Prepare for predictions

### Add column which states whether a bike was added at a given moment

In [96]:
df['bike_added'] = df.apply(lambda x: df.iloc[(x.name - 1 if x.name > 0 else 0)]['bikes'] < x['bikes'] , axis=1)
df['bike_removed'] = df.apply(lambda x: df.iloc[(x.name - 1 if x.name > 0 else 0)]['bikes'] > x['bikes'] , axis=1)
df.head(100)

,index,ts,lat,lon,operative,sid,name,bikes,total_slots,hour_and_minutes,month,bikes_percent,minutes_from_midnight,hour,date,weekday,weekend,bike_added,bike_removed
0,0,2017-03-29T08:26:01,60.202791,24.966613,1,D21,Arabian kauppakeskus,0,1,08:26,03,0.0,506,08,2017-03-29 08:26:01,2,False,False,False
1,1,2017-03-29T08:27:01,60.202791,24.966613,1,D21,Arabian kauppakeskus,0,1,08:27,03,0.0,507,08,2017-03-29 08:27:01,2,False,False,False
2,2,2017-03-29T08:28:01,60.202791,24.966613,1,D21,Arabian kauppakeskus,0,1,08:28,03,0.0,508,08,2017-03-29 08:28:01,2,False,False,False
3,3,2017-03-29T08:29:01,60.202791,24.966613,1,D21,Arabian kauppakeskus,0,1,08:29,03,0.0,509,08,2017-03-29 08:29:01,2,False,False,False
4,4,2017-03-29T08:30:01,60.202791,24.966613,1,D21,Arabian kauppakeskus,0,1,08:30,03,0.0,510,08,2017-03-29 08:30:01,2,False,False,False
5,5,2017-03-29T08:31:01,60.202791,24.966613,1,D21,Arabian kauppakeskus,0,1,08:31,03,0.0,511,08,2017-03-29 08:31:01,2,False,False,False
6,6,2017-03-29T08:32:01,60.202791,24.966613,1,D21,Arabian kauppakeskus,0,1,08:32,03,0.0,512,08,2017-03-29 08:32:01,2,False,False,False
7,7,2017-03-29T08:33:01,60.202791,24.966613,1,D21,Arabian kauppakeskus,0,1,08:33,03,0.0,513,08,2017-03-29 08:33:01,2,False,False,False
8,8,2017-03-29T08:34:01,60.202791,24.966613,1,D21,Arabian kauppakeskus,0,1,08:34,03,0.0,514,08,2017-03-29 08:34:01,2,False,False,False
9,9,2017-03-29T08:35:01,60.202791,24.966613,1,D21,Arabian kauppakeskus,0,1,08:35,03,0.0,515,08,2017-03-29 08:35:01,2,False,False,False


### Add max waiting time for next bike

In [97]:
# extract difference in minutes to a separate column
# on the moment that a bike was added, the waiting time is the time to next bike
i = 0
first_non_additive_index = 0
while (i < len(df)):
    while (i < len(df) and df.iloc[i]['bike_added'] == False):
        i += 1
    if (i >= len(df)):
        break
    additive_row = df.iloc[i]
#     time = additive_row['minutes_from_midnight']
#     difference = abs(time - df.iloc[first_non_additive_index]['minutes_from_midnight'])
#     df.loc[first_non_additive_index:(i-1 if i > 0 else 0), 'max_wait_for_new_bike'] = difference 
    # add time using a date object (more accurate if wait for more than 24h)
    date_time = additive_row['date']
    last_date = df.iloc[first_non_additive_index]['date']
    date_difference = abs((date_time - last_date).total_seconds() / 60)
    df.loc[first_non_additive_index:(i-1 if i > 0 else 0), 'max_wait_for_new_bike_as_date'] = date_difference 
    first_non_additive_index = i
    i += 1
# if we can't know when the next bike will be added, fill with -1 for now
df = df.fillna(-1)
df.head(100)

,index,ts,lat,lon,operative,sid,name,bikes,total_slots,hour_and_minutes,month,bikes_percent,minutes_from_midnight,hour,date,weekday,weekend,bike_added,bike_removed,max_wait_for_new_bike_as_date
0,0,2017-03-29T08:26:01,60.202791,24.966613,1,D21,Arabian kauppakeskus,0,1,08:26,03,0.0,506,08,2017-03-29 08:26:01,2,False,False,False,50730.0
1,1,2017-03-29T08:27:01,60.202791,24.966613,1,D21,Arabian kauppakeskus,0,1,08:27,03,0.0,507,08,2017-03-29 08:27:01,2,False,False,False,50730.0
2,2,2017-03-29T08:28:01,60.202791,24.966613,1,D21,Arabian kauppakeskus,0,1,08:28,03,0.0,508,08,2017-03-29 08:28:01,2,False,False,False,50730.0
3,3,2017-03-29T08:29:01,60.202791,24.966613,1,D21,Arabian kauppakeskus,0,1,08:29,03,0.0,509,08,2017-03-29 08:29:01,2,False,False,False,50730.0
4,4,2017-03-29T08:30:01,60.202791,24.966613,1,D21,Arabian kauppakeskus,0,1,08:30,03,0.0,510,08,2017-03-29 08:30:01,2,False,False,False,50730.0
5,5,2017-03-29T08:31:01,60.202791,24.966613,1,D21,Arabian kauppakeskus,0,1,08:31,03,0.0,511,08,2017-03-29 08:31:01,2,False,False,False,50730.0
6,6,2017-03-29T08:32:01,60.202791,24.966613,1,D21,Arabian kauppakeskus,0,1,08:32,03,0.0,512,08,2017-03-29 08:32:01,2,False,False,False,50730.0
7,7,2017-03-29T08:33:01,60.202791,24.966613,1,D21,Arabian kauppakeskus,0,1,08:33,03,0.0,513,08,2017-03-29 08:33:01,2,False,False,False,50730.0
8,8,2017-03-29T08:34:01,60.202791,24.966613,1,D21,Arabian kauppakeskus,0,1,08:34,03,0.0,514,08,2017-03-29 08:34:01,2,False,False,False,50730.0
9,9,2017-03-29T08:35:01,60.202791,24.966613,1,D21,Arabian kauppakeskus,0,1,08:35,03,0.0,515,08,2017-03-29 08:35:01,2,False,False,False,50730.0


### Add idle time for each station

In [98]:
i = 0
idle_for = 0
idle_for_as_date = 0
while (i < len(df)):
    row = df.iloc[i]
    previous_row = df.iloc[(i-1 if i > 0 else 0)]
    if (not row['bikes'] == previous_row['bikes']):
        idle_for = 0
        idle_for_as_date = 0
        df.loc[i, 'idle_for'] = idle_for
        df.loc[i, 'idle_for_as_date'] = idle_for_as_date        
        i+= 1
        continue
#     time_now = row['minutes_from_midnight']
#     time_last = previous_row['minutes_from_midnight']
#     idle_for += abs(time_now - time_last)
#     df.loc[i, 'idle_for'] = idle_for
    # add time using a date object (more accurate if wait for more than 24h)
    date_time_now = row['date']
    date_time_last = previous_row['date']
    secs = (date_time_now - date_time_last).total_seconds()
    idle_for_as_date += abs(secs / 60)
    df.loc[i, 'idle_for_as_date'] = idle_for_as_date
    i += 1
# if we can't know when the next bike will be taken, fill with -1 for now
df = df.fillna(-1)
df.head(50)

,index,ts,lat,lon,operative,sid,name,bikes,total_slots,hour_and_minutes,...,minutes_from_midnight,hour,date,weekday,weekend,bike_added,bike_removed,max_wait_for_new_bike_as_date,idle_for_as_date,idle_for
0,0,2017-03-29T08:26:01,60.202791,24.966613,1,D21,Arabian kauppakeskus,0,1,08:26,...,506,08,2017-03-29 08:26:01,2,False,False,False,50730.0,0.000000,-1.0
1,1,2017-03-29T08:27:01,60.202791,24.966613,1,D21,Arabian kauppakeskus,0,1,08:27,...,507,08,2017-03-29 08:27:01,2,False,False,False,50730.0,1.000000,-1.0
2,2,2017-03-29T08:28:01,60.202791,24.966613,1,D21,Arabian kauppakeskus,0,1,08:28,...,508,08,2017-03-29 08:28:01,2,False,False,False,50730.0,2.000000,-1.0
3,3,2017-03-29T08:29:01,60.202791,24.966613,1,D21,Arabian kauppakeskus,0,1,08:29,...,509,08,2017-03-29 08:29:01,2,False,False,False,50730.0,3.000000,-1.0
4,4,2017-03-29T08:30:01,60.202791,24.966613,1,D21,Arabian kauppakeskus,0,1,08:30,...,510,08,2017-03-29 08:30:01,2,False,False,False,50730.0,4.000000,-1.0
5,5,2017-03-29T08:31:01,60.202791,24.966613,1,D21,Arabian kauppakeskus,0,1,08:31,...,511,08,2017-03-29 08:31:01,2,False,False,False,50730.0,5.000000,-1.0
6,6,2017-03-29T08:32:01,60.202791,24.966613,1,D21,Arabian kauppakeskus,0,1,08:32,...,512,08,2017-03-29 08:32:01,2,False,False,False,50730.0,6.000000,-1.0
7,7,2017-03-29T08:33:01,60.202791,24.966613,1,D21,Arabian kauppakeskus,0,1,08:33,...,513,08,2017-03-29 08:33:01,2,False,False,False,50730.0,7.000000,-1.0
8,8,2017-03-29T08:34:01,60.202791,24.966613,1,D21,Arabian kauppakeskus,0,1,08:34,...,514,08,2017-03-29 08:34:01,2,False,False,False,50730.0,8.000000,-1.0
9,9,2017-03-29T08:35:01,60.202791,24.966613,1,D21,Arabian kauppakeskus,0,1,08:35,...,515,08,2017-03-29 08:35:01,2,False,False,False,50730.0,9.000000,-1.0


In [99]:
df.describe()

,index,lat,lon,operative,bikes,total_slots,bikes_percent,minutes_from_midnight,weekday,max_wait_for_new_bike_as_date,idle_for_as_date,idle_for
count,320175.000000,3.201750e+05,3.201750e+05,320175.0,320175.000000,320175.000000,320175.000000,320175.000000,320175.000000,320175.000000,320175.000000,320175.000000
mean,160434.329557,6.020279e+01,2.496661e+01,1.0,15.385645,21.643130,0.706714,738.822406,3.019689,1126.679173,95.410123,-0.883842
std,93025.705319,5.069091e-10,2.354387e-11,0.0,9.221420,4.787947,0.387949,391.821543,1.980844,16219.981090,2609.043310,0.320415
min,0.000000,6.020279e+01,2.496661e+01,1.0,0.000000,1.000000,0.000000,0.000000,0.000000,-1.000000,0.000000,-1.000000
25%,80043.500000,6.020279e+01,2.496661e+01,1.0,9.000000,24.000000,0.416667,412.000000,1.000000,16.000000,2.000000,-1.000000
50%,160087.000000,6.020279e+01,2.496661e+01,1.0,15.000000,24.000000,0.666667,741.000000,3.000000,39.000000,8.000000,-1.000000
75%,240130.500000,6.020279e+01,2.496661e+01,1.0,21.000000,24.000000,0.916667,1071.000000,5.000000,115.000000,26.000000,-1.000000
max,335087.000000,6.020279e+01,2.496661e+01,1.0,55.000000,24.000000,2.291667,1439.000000,6.000000,275737.000000,202986.000000,0.000000


### Add time for next bike take

Predict how much time for the next bike to be taken: max waiting time is not useful here.

In [109]:
i = 0
first_non_removing_index = 0
while (i < len(df)):
    while (i < len(df) and df.iloc[i]['bike_removed'] == False):
        i += 1
    if (i >= len(df)):
        break
    removing_row = df.iloc[i]
#     time = removing_row['minutes_from_midnight']
#     times_until_now = df.loc[first_non_removing_index:i-1, 'minutes_from_midnight']
#     difference = abs(time - times_until_now)
#     df.loc[first_non_removing_index:(i-1 if i > 0 else 0), 'wait_for_bike_taken'] = difference 
    # add time using a date object (more accurate if wait for more than 24h)
    date_time = removing_row['date']
    date_times_until_now = df.loc[first_non_removing_index:i-1, 'date']
    date_difference = abs((date_time - date_times_until_now).apply(lambda x: x.total_seconds() / 60))
    df.loc[first_non_removing_index:(i-1 if i > 0 else 0), 'wait_for_bike_taken_as_date'] = date_difference 
    first_non_removing_index = i
    i += 1
# if we can't know when the next bike will be taken, fill with -1 for now
df = df.fillna(-1)
df.head(50)

,index,ts,lat,lon,operative,sid,name,bikes,total_slots,hour_and_minutes,...,hour,date,weekday,weekend,bike_added,bike_removed,max_wait_for_new_bike_as_date,idle_for_as_date,idle_for,wait_for_bike_taken_as_date
0,0,2017-03-29T08:26:01,60.202791,24.966613,1,D21,Arabian kauppakeskus,0,1,08:26,...,08,2017-03-29 08:26:01,2,False,False,False,50730.0,0.000000,-1.0,50737.000000
1,1,2017-03-29T08:27:01,60.202791,24.966613,1,D21,Arabian kauppakeskus,0,1,08:27,...,08,2017-03-29 08:27:01,2,False,False,False,50730.0,1.000000,-1.0,50736.000000
2,2,2017-03-29T08:28:01,60.202791,24.966613,1,D21,Arabian kauppakeskus,0,1,08:28,...,08,2017-03-29 08:28:01,2,False,False,False,50730.0,2.000000,-1.0,50735.000000
3,3,2017-03-29T08:29:01,60.202791,24.966613,1,D21,Arabian kauppakeskus,0,1,08:29,...,08,2017-03-29 08:29:01,2,False,False,False,50730.0,3.000000,-1.0,50734.000000
4,4,2017-03-29T08:30:01,60.202791,24.966613,1,D21,Arabian kauppakeskus,0,1,08:30,...,08,2017-03-29 08:30:01,2,False,False,False,50730.0,4.000000,-1.0,50733.000000
5,5,2017-03-29T08:31:01,60.202791,24.966613,1,D21,Arabian kauppakeskus,0,1,08:31,...,08,2017-03-29 08:31:01,2,False,False,False,50730.0,5.000000,-1.0,50732.000000
6,6,2017-03-29T08:32:01,60.202791,24.966613,1,D21,Arabian kauppakeskus,0,1,08:32,...,08,2017-03-29 08:32:01,2,False,False,False,50730.0,6.000000,-1.0,50731.000000
7,7,2017-03-29T08:33:01,60.202791,24.966613,1,D21,Arabian kauppakeskus,0,1,08:33,...,08,2017-03-29 08:33:01,2,False,False,False,50730.0,7.000000,-1.0,50730.000000
8,8,2017-03-29T08:34:01,60.202791,24.966613,1,D21,Arabian kauppakeskus,0,1,08:34,...,08,2017-03-29 08:34:01,2,False,False,False,50730.0,8.000000,-1.0,50729.000000
9,9,2017-03-29T08:35:01,60.202791,24.966613,1,D21,Arabian kauppakeskus,0,1,08:35,...,08,2017-03-29 08:35:01,2,False,False,False,50730.0,9.000000,-1.0,50728.000000


In [101]:
max_wait_counts = df['max_wait_for_new_bike_as_date'].value_counts()
wait_counts = df['wait_for_bike_taken_as_date'].value_counts()
print(max_wait_counts)
print(wait_counts)

 12.000000     5701
 8.000000      5689
 14.000000     5573
 13.000000     5481
 7.000000      5426
 6.000000      5396
 9.000000      5380
 10.000000     5377
 5.000000      5208
 15.000000     5080
 4.000000      5022
 11.000000     5012
 16.000000     5010
 17.000000     4966
 20.000000     4537
 18.000000     4419
 19.000000     4356
 3.000000      4203
 22.000000     4130
 24.000000     3917
 25.000000     3799
 21.000000     3798
 27.000000     3633
 26.000000     3614
 23.000000     3396
 30.000000     3363
 31.000000     3332
 29.000000     3167
 28.000000     3060
 2.000000      3014
               ... 
 44.016667       44
 21.983333       44
 21.016667       42
 42.016667       42
 39.983333       40
 40.933333       40
 40.016667       39
 37.983333       38
 36.983333       37
 35.983333       36
 1.016667        36
 0.983333        36
 17.983333       36
 34.983333       35
 35.016667       35
 16.983333       34
 32.983333       33
 16.016667       32
 30.016667       30


## Remove useless or codependent columns

In [102]:
store_df = df
cleaned_df = df.drop(columns=['index', 'ts', 'sid', 'hour_and_minutes', 'name', 'bike_added', 'bike_removed', 'lat', 'lon', 'bikes', 'total_slots', 'date'])
filterable_max_waits = max_wait_counts[max_wait_counts < 10].index
filterable_waits = wait_counts[wait_counts < 10].index
cleaned_df = cleaned_df[~cleaned_df['max_wait_for_new_bike_as_date'].isin(filterable_max_waits)]
cleaned_df = cleaned_df[~cleaned_df['wait_for_bike_taken_as_date'].isin(filterable_waits)]
print('removed', len(df) + len(cleaned_df), 'rows')
print('using columns', cleaned_df.columns.values)

removed 636554 rows
using columns ['operative' 'month' 'bikes_percent' 'minutes_from_midnight' 'hour'
 'weekday' 'weekend' 'max_wait_for_new_bike_as_date' 'idle_for_as_date'
 'idle_for' 'wait_for_bike_taken_as_date']


## Add classes for prediciton

In [103]:
def define_class(x):
    if x < 5:
        return 0
    if x < 100:
        return 1
    return 2

cleaned_df['max_wait_for_new_bike_class'] = cleaned_df['max_wait_for_new_bike_as_date'].apply(define_class)
cleaned_df['max_wait_for_bike_taken_class'] = cleaned_df['wait_for_bike_taken_as_date'].apply(define_class)
print('value count of classes in wait for next bike taken:')
print(cleaned_df['max_wait_for_bike_taken_class'].value_counts())
print()
print('value count of classes in max wait for new bike')
print(cleaned_df['max_wait_for_new_bike_class'].value_counts())

value count of classes in wait for next bike taken:
1    202194
0     61203
2     52982
Name: max_wait_for_bike_taken_class, dtype: int64

value count of classes in max wait for new bike
1    216182
2     85356
0     14841
Name: max_wait_for_new_bike_class, dtype: int64


## Define training and test sets

In [104]:
# drop the columns we would not know in a real situation, as well as the column to be predicted
X = cleaned_df.drop(columns=['max_wait_for_new_bike_class', 'max_wait_for_new_bike_as_date', 'wait_for_bike_taken_as_date']).as_matrix()
y = cleaned_df['max_wait_for_bike_taken_class'].as_matrix()
# set 
train_size = int(len(X)*0.8)

# define the training and test sets
X_train = torch.from_numpy(np.asarray(X[:train_size].astype('float')))
# take transpose to get right dimensions for torch
y_train = torch.from_numpy(np.asarray([y[:train_size]]).astype('float').T)
X_test = torch.from_numpy(np.asarray(X[train_size:].astype('float')))
y_test = torch.from_numpy(np.asarray([y[train_size:]]).astype('float').T)
# combine input and labels
train_data = torch.cat((X_train, y_train), 1)
test_data = torch.cat((X_test, y_test), 1)

# sanity check
assert len(X) == len(X_train) + len(X_test)

## Define a simple neural network

In [134]:
class Net(nn.Module):
    def __init__(self, in_neurons=10, out_neurons=3, nr_layers=1, nr_neurons=10):
        super(Net, self).__init__()
        self.layers = nn.ModuleList()
        for i in range(nr_layers):
            self.layers.append(nn.Linear(in_neurons, nr_neurons))
            in_neurons = nr_neurons
        self.output_layer = nn.Linear(in_neurons, out_neurons)

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
            x = funcs.relu(x)
        return self.output_layer(x)

## Define loss function

In [106]:
loss_func = torch.nn.CrossEntropyLoss()

## Define training loop

In [130]:
net = Net().double()
optimizer = torch.optim.Adam(net.parameters(), lr=0.05)
data_len = len(train_data)
batch_size = 250
epochs = 50
for epoch in range(epochs):
    # shuffle the data
    indices = torch.randperm(data_len)
    # train the data in batches
    for i in range(0, data_len, batch_size):
        batch_indices = indices[i: i+ batch_size]
        data = train_data[batch_indices]
        # assume that last cell in array is label
        inp, label = data[:, :data.shape[1]-1], data[:, data.shape[1]-1]
        # torch has to be fed variables
        input_var = Variable(inp)
        label_var = Variable(label.long())
        # reset parameters after each training round
        optimizer.zero_grad()
        # actual forward pass
        outputs = net(input_var)
        # calculate loss
        loss = loss_func(outputs, label_var)
        loss.backward()
        optimizer.step()
    if epoch % 10 == 0:
        print('e:', epoch)
        print('loss', loss.item())
        print('--')

print('training done with a training set of length', len(train_data))

e: 0
loss 0.9659995593325107
--
e: 10
loss 0.9050406739159506
--
e: 20
loss 0.8991931131788318
--
e: 30
loss 0.9182867547258734
--
e: 40
loss 0.8643191567236812
--
training done with a training set of length 253103


## Evaluate with test set

In [133]:
total_loss = 0
inp, label = test_data[:, :test_data.shape[1]-1], test_data[:, test_data.shape[1]-1]
test_input_var = Variable(inp)
test_label_var = Variable(label.long())
test_outputs = net(test_input_var)
# just print out the loss for now
test_loss = loss_func(test_outputs, test_label_var)
# this is basically just to see whether the net always predicted the same thing
total_loss = test_loss.item()
probs = funcs.softmax(test_outputs, dim=1)
predictions = torch.round(funcs.softmax(test_outputs, dim=1))
i = 0
for true, pred in zip(test_label_var.data, funcs.softmax(test_outputs, dim=1).data):
    if i == 100:
        break
    print(true, pred)
    i += 1

tensor(1) tensor([ 0.1771,  0.6490,  0.1738], dtype=torch.float64)
tensor(1) tensor([ 0.1771,  0.6490,  0.1738], dtype=torch.float64)
tensor(1) tensor([ 0.1771,  0.6490,  0.1738], dtype=torch.float64)
tensor(1) tensor([ 0.1771,  0.6490,  0.1738], dtype=torch.float64)
tensor(1) tensor([ 0.1771,  0.6490,  0.1738], dtype=torch.float64)
tensor(1) tensor([ 0.1771,  0.6490,  0.1738], dtype=torch.float64)
tensor(1) tensor([ 0.1771,  0.6490,  0.1738], dtype=torch.float64)
tensor(1) tensor([ 0.1771,  0.6490,  0.1738], dtype=torch.float64)
tensor(1) tensor([ 0.1771,  0.6490,  0.1738], dtype=torch.float64)
tensor(1) tensor([ 0.1771,  0.6490,  0.1738], dtype=torch.float64)
tensor(1) tensor([ 0.1771,  0.6490,  0.1738], dtype=torch.float64)
tensor(1) tensor([ 0.1771,  0.6490,  0.1738], dtype=torch.float64)
tensor(1) tensor([ 0.1771,  0.6490,  0.1738], dtype=torch.float64)
tensor(1) tensor([ 0.1771,  0.6490,  0.1738], dtype=torch.float64)
tensor(1) tensor([ 0.1771,  0.6490,  0.1738], dtype=torch.floa